In [2]:
import os
from openai import OpenAI
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, List, Optional, Tuple, Union

import PyPDF2
import markdown
import html2text
import json
from tqdm import tqdm
import tiktoken
import re
from bs4 import BeautifulSoup
from IPython.display import display, Code, Markdown


In [2]:

api_key = 'your api'
base_url = "your url"  


In [3]:
# 实例化客户端
client = OpenAI(api_key=api_key, base_url = base_url)

In [4]:
# 临时设置环境变量
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_BASE_URL"] = base_url

# 1.RAG技术原理

### 1.1 RAG技术必要性介绍

LLM生成语言时，产生幻觉或不准确的结果

 - 信息误导：过时或不准确
 - 知识更新之后：无法访问新的技术
 - 推理能力不足：LLM在面对复杂推理任务时，难以提供高效且准确的结果
 
 ### 1.2 RAG系统的核心组件说明

 1. **向量化模块：**用于将文档片段转化为向量表示，以便后续检索
 2. **文档加载与切分模块：**负责加载文档并将其切分为若干易于处理的文档片段
 3. **数据库模块：**用于存储文档片段及其对应的向量表示
 4. **检索模块：**根据用户输入的查询，从数据库中检索最相关的文档片段
 5. **生成模块：**将检索到的文档片段与用户输入的查询结合，生成最终的回答

 ### 1.3 RAG系统的基本流程：

 - **索引：**将文本库分割成焦段的Chunk，并通过编码器构建向量库索引
 - **检索：**根据问题和chunks的相似度检索相关文档片段
 - **生成：**一检索到的上下文为条件，生成问题的回答

# 2. OpenAI第三档embedding模型介绍与调用方法

### 2.1 OpenAI Embedding模型介绍

将文本字符串表示为向量（浮点数列表），通过计算向量之间的距离来衡量文本之间的相关性，向量距离越小，表示文本之间的相关性越高；反之越低。常见embedding应用包括：

 - 搜索：根据文本查询的相关性对结果进行排序
 - 聚类：根据文本相似性将其分组
 - 推荐：根据相关文本字符串推荐项目
 - 异常检测：识别与其他内容相关性较低的异常点
 - 多样性测量：分析相似性分布
 - 分类：将文本字符串根据其最相似的标签进行分类

OpenAI最新的embedding模型是text-embedding-3-small和text-embedding-3-large向量长度分别为1536和3072的向量。用户可以设置维度参数来介绍向量的维度，不损失其表达概念的能力   

### 2.2OpenAI Embedding模型获取方法（付费）：
要获取文本的Embedding向量，可以将文本字符串发送到OpenAI的EmbeedingAPI断点，并指定所使用的模型

这里用阿里云百炼embedding的模型替代

In [5]:
# 调用embedding API 获取文本的向量表示
response = client.embeddings.create(
    input = "测试文本",
    model = "text-embedding-v3"
)

In [6]:
print(response.data[0].embedding)

[-0.10152608901262283, 0.043125443160533905, -0.03668363019824028, -0.012690761126577854, -0.027599511668086052, -0.028891732916235924, 0.018428605049848557, 0.07907617092132568, -0.024976497516036034, 0.016885655000805855, -0.007864221930503845, 0.028949594125151634, -0.010250971652567387, 0.007377228233963251, -0.004681888036429882, -0.02233419567346573, 0.028756724670529366, -0.029084600508213043, -0.011822852306067944, -0.08208492398262024, -0.07398443669080734, 0.008481401950120926, 0.03436920419335365, 0.044861260801553726, 0.02954748645424843, 0.08725380152463913, -0.058169201016426086, -0.07340583205223083, 0.016577064990997314, -0.010183467529714108, 0.0658068060874939, 0.015622365288436413, 0.03924878314137459, -0.053578928112983704, -0.019807616248726845, -0.0008956340607255697, 0.005959643051028252, -0.03226693347096443, -0.06954845786094666, 0.00078714539995417, 0.041389625519514084, -0.03919092193245888, 0.0005560043500736356, -0.030183952301740646, -0.005626944825053215,

In [7]:
len(response.data[0].embedding)

1024

余弦相似度与效果介绍

$$
\text { Cosine Similarity }(\vec{a}, \vec{b})=\frac{\vec{a} \cdot \vec{b}}{\|\vec{a}\|\|\vec{b}\|}
$$

In [8]:
def consine_sim(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude = np.linalg.norm(v1) * np.linalg.norm(v2)
    if not magnitude:
        return 0
    return dot_product / magnitude

In [9]:
text1 = '我喜欢吃苹果'
text2 = '苹果是我最喜欢吃的水果'
text3 = '我喜欢用苹果手机'

In [10]:
vector1 = client.embeddings.create(
    input = text1,
    model = "text-embedding-v3"
).data[0].embedding

vector2 = client.embeddings.create(
    input = text2,
    model = "text-embedding-v3"
).data[0].embedding

vector3 = client.embeddings.create(
    input = text3,
    model = "text-embedding-v3"
).data[0].embedding

In [11]:
consine_sim(vector1, vector2)

0.8627496458683991

In [12]:
consine_sim(vector1, vector3)

0.848912233853192

In [13]:
consine_sim(vector2, vector3)

0.7528024373146859

根据实验，embedding模型可以根据句意相关的embedding的处理

In [14]:
class BaseEmbeddings:
    """
    向量化的基类，用于将文本转化为向量表示，不同的子类可以实现不同的向量获取方式
    """

    def __init__(self, path: str, is_api: bool) -> None:
        """
        初始化基类

        参数：
        path(str) - 如果是本地模型， path 表示模型路径；如果是api模式，则path可以为空
        is_api(bool) - 是否使用api模式
        """
        self.path = path
        self.is_api = is_api

    def get_embedding(self, text:str, model: str) -> List[float]:
        """
        抽象方法：用于获取文本的向量表示

        参数：

        text(str) - 待获取向量表示的文本
        model(str) - 模型名称

        返回值：

        List[float] - 文本的向量表示
        """
        raise NotImplementedError("子类必须实现该方法")
        
    @classmethod
    def cosine_similarity(cls, vector1: List[float], vector2: List[float]) -> float:
        """
        计算两个向量之间的余弦相似度，用于衡量他们的相似程度

        参数：
        vector1（list[float]）- 第一个向量
        vector2（list[float]）- 第二个向量
        
        返回：
        float - 预选相似度值，范围从 -1 到 1，越接近 1 表示向量越相似
        """
        dot_product = np.dot(vector1, vector2)
        magnitude = np.linalg.norm(vector1) * np.linalg.norm(vector2)
        if not magnitude:
            return 0
        return dot_product / magnitude

In [15]:
class OpenAIEmbedding(BaseEmbeddings):
    """
    使用 OpenAI 的 Embedding API 来获取文本向量的类，继承自BaseEmbeddings
    """

    def __init__(self, path: str = '', is_api: bool = True) -> None:
        """
        初始化类，设置 OpenAI API 客户端，如果使用的是 API 调用

        参数：
        path(str) - 如果是本地模型， path 表示模型路径；如果是api模式，则path可以为空
        is_api(bool) - 是否使用api模式
        """
        super().__init__(path, is_api)
        if self.is_api:
            # 初始化 OpenAI API 客户端
            self.client = OpenAI()
            self.client.api_key = os.getenv('OPENAI_API_KEY')
            self.client.base_url = os.getenv('OPENAI_BASE_URL')

    def get_embedding(self, text: str, model: str = 'text-embedding-v3') -> List[float]:
        """
        使用 Open API 获取文本的向量表示

        参数：
        text(str) - 待获取向量表示的文本
        model(str) - 模型名称

        返回值：
        List[float] - 文本的向量表示
        """
        if self.is_api:
            text = text.replace('\n', ' ')
            return self.client.embeddings.create(input=[text], model=model).data[0].embedding
        else:
            raise NotImplementedError("本地模型尚未实现")

In [16]:
embedding_model = OpenAIEmbedding()
text = "这是一个实例文本，用于演示 OpenAI Embedding 的使用。"
embedding_vector = embedding_model.get_embedding(text)
print("文本的向量表示为：", embedding_vector)

文本的向量表示为： [-0.05204948037862778, 0.023631660267710686, -0.04561807960271835, -0.0034751067869365215, -0.0711941123008728, -0.009983628988265991, 0.029576966539025307, 0.10701552778482437, 0.022528599947690964, 0.0014080374967306852, -0.010731466114521027, 0.015517625026404858, 0.054741695523262024, 0.02159380353987217, 0.030549155548214912, -0.02267816849052906, 0.019761601462960243, 0.011105384677648544, 0.03438182175159454, -0.0665949136018753, -0.07082019746303558, -0.002897869795560837, -0.013161937706172466, 0.0005871691973879933, 0.03135308250784874, 0.08218732476234436, -0.0292404405772686, -0.04262672737240791, 0.022939911112189293, -0.07104454934597015, 0.036326199769973755, -0.0027576503343880177, 0.026118218898773193, -0.028660867363214493, 0.00014511258632410318, -0.00080801488365978, 0.02204250544309616, -0.022135986015200615, -0.11554086953401566, 0.004874964710325003, 0.03213831037282944, -0.025538645684719086, -0.0029796643648296595, -0.017209606245160103, 0.00154241453

In [17]:
vector1 = embedding_model.get_embedding(text1)
vector2 = embedding_model.get_embedding(text2)
similarity = embedding_model.cosine_similarity(vector1, vector2)
print(f"两段文本的余弦相似度为：{similarity}")

两段文本的余弦相似度为：0.8627496458683991


# 3.文档加载与切分模块创建

### 3.1 文档格式处理函数

展示支持多种格式的简单实现：

In [18]:
def read_file_content(cls, file_path: str):
    # 根据文件扩展名选择读取方式
    if file_path.endswith('.pdf'):
        return cls.read_pdf(file_path)
    elif file_path.endswith('.md'):
        return cls.read_markdown(file_path)
    elif file_path.endswith('.txt'):
        return cls.read_text(file_path)
    else:
        raise ValueError('Unsupported file type')

# 3.2 文档切分函数

按照token长度进行切分，设置一个最大Token长度，然后按照这个长度进行切分。在这个过程我们也会保证每个片段之间有一定的重叠，避免重要信息被切掉。

In [19]:
def get_chunk(cls, text:str, max_token_len:int = 600, cover_content:int=150):
    chunk_text = []
    curr_len = 0
    curr_chunk = ''
    lines = text.split('\n')

    for line in lines:
        line = line.replace(' ', '')
        line_len = len(enc.encode(line))
        if line_len > max_token_len:
            print('warning line_len = ', line_len)
        if curr_len + line_len <= max_token_len:
            curr_chunl += line + '\n'
            curr_len += line_len + 1
        else:
            chunk_text.append(curr_chunk)
            curr_chunk = curr_chunk[-cover_content:] + line
            curr_len = len(enc.encode(curr_chunk))

    if curr_chunk:
        chunk_text.append(curr_chunk)
    
    return chunk_text

In [20]:
# 需要梯子
enc = tiktoken.get_encoding('cl100k_base')
len(enc.encode('你好啊,好久不见！'))

11

In [21]:
class ReadFiles:
    """
    读取文件类，用于从指定路径读取支持的文件类型，并进行切分
    当前支持类型：pdf, md, txt
    """

    def __init__(self, path:str) -> None:
        """
        初始化函数，设定要读取的文件路径，并获取该路径下所有符合要求的文件。
        ：param path：文件夹路径
        """
        self._path = path
        self.file_list = self.get_files() 

    def get_files(self):
        """
        遍历指定文件夹，获取支持的文件类型列表（txt, md, pdf）
        return 文件路径列表
        """
        file_list = []
        for filepath, dirnames, filenames in os.walk(self._path):
            for filename in filenames:
                if filename.endswith('.md'):
                    file_list.append(os.path.join(filepath, filename))
                elif filename.endswith('.pdf'):
                    file_list.append(os.path.join(filepath, filename))
                elif filename.endswith('.txt'):
                    file_list.append(os.path.join(filepath, filename))
        return file_list
    
    def get_content(self, max_token_len:int = 600, cover_content: int = 150):
        """
        读取文件内容并进行分割，将长文本切分为多个块
        param max_token_len: 每个文档片段最大 Token 长度
        prarm cover_content: 每个文档片段之间的 Token 长度
        return： 切分后的文档片段列表
        """
        docs = []
        for file in self.file_list:
            content = self.read_file_content(file)
            chunk_content = self.get_chunk(content, max_token_len=max_token_len, cover_content=cover_content)
            docs.extend(chunk_content)
        return docs

    @classmethod
    def get_chunk(cls, text:str, max_token_len:int = 600, cover_content:int=150):
        chunk_text = []
        curr_len = 0
        curr_chunk = ''
        token_len = max_token_len - cover_content
        lines = text.splitlines()

        for line in lines:
            line = line.replace(' ', '')
            line_len = len(enc.encode(line))
            if line_len > max_token_len:
                # 如果但凡长度超过限制，将其分给为多个片段
                num_chunks = (line_len + token_len - 1) // token_len
                for i in range(num_chunks):
                    start = i * token_len
                    end = start + token_len
                    # 防止跨单词分割
                    while not line[start:end].rstrip().isspace():
                        start += 1
                        end += 1
                        if start >= line_len:
                            break
                    curr_chunk = curr_chunk[-cover_content:] + line[start:end]
                    chunk_text.append(curr_chunk)
                start = (num_chunks - 1) * token_len
                curr_chunk = curr_chunk[-cover_content:] + line[start:end]
                chunk_text.append(curr_chunk)
            elif curr_len + line_len <= max_token_len:
                # 当前片段长度未超过限制时，继续累加
                curr_chunk += line + '\n'
                curr_len += line_len + 1
            else:    
                chunk_text.append(curr_chunk)
                curr_chunk = curr_chunk[-cover_content:] + line
                curr_len = len(enc.encode(curr_chunk))

        if curr_chunk:
            chunk_text.append(curr_chunk)
        
        return chunk_text
    
    @classmethod
    def read_file_content(cls, file_path:str):
        """
        读取文件内容，根据文件类型选择不同的读取方式
        param file_path: 文件路径
        return： 文件内容
        """

        if file_path.endswith('.pdf'):
            return cls.read_pdf(file_path)
        elif file_path.endswith('.md'):
            return cls.read_markdown(file_path)
        elif file_path.endswith('.txt'):
            return cls.read_text(file_path)
        else:
            raise ValueError('Unsupported file type')
        
    @classmethod
    def read_pdf(cls, file_path:str):
        """
        读取pdf文件内容
        param file_path: 文件路径
        return： 文件内容
        """
        with open(file_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            text = ''
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
            return text

    @classmethod
    def read_markdown(cls, file_path:str):
        """
        读取markdown文件内容
        param file_path: 文件路径
        return： 文件内容
        """
        with open(file_path, 'r', encoding='utf-8') as f:
           md_text = f.read()
           html_text = markdown.markdown(md_text)
           # 使用BeautifulSoup（html_text, 'html.parser'）
           soup = BeautifulSoup(html_text, 'html.parser')
           plain_text = soup.get_text()
           # 使用正则表达式一次网址链接
           text = re.sub(r'http\S+', '', plain_text)
           return text
    
    @classmethod
    def read_text(cls, file_path:str):
        """
        读取txt文件内容
        param file_path: 文件路径
        return： 文件内容
        """
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()

In [22]:
class Documents:
    """
    文档类：用于读取已分好类的json格式文档
    """
    def __init__(self, path:str = '') -> None:
        self.path = path
    
    def get_content(self):
        """
        读取 json 格式的文档内容
        return json 文档的内容
        """
        with open(self.path, 'r', encoding='utf-8') as f:
            content = json.load(f)
        return content

In [ ]:
# 初始化 ReadFiles类，指定文件目录路径data
file_reader = ReadFiles('data')

# 获取目录下所有支持的文件类型
file_list = file_reader.get_files()
print("支持的文件类型：", file_list)

In [24]:
document_chunks = file_reader.get_content(max_token_len=600, cover_content=150)
print("分块后的文档内容：", document_chunks)

分块后的文档内容： ['金融序列预测\n\n挑战与任务：时间序列与自然语言对齐\n该方法与传统训练方法比较：\n\n模型介绍：\n\ninstanceNorm\npatching：获取每一小部分的时间序列窗口，移动窗口，聚合局部特征\npatchembedder：批处理向量化\ntextprototypes：先将预训练好的embedding与linear连接在一起，得到形状更小的embedding(prototypes翻译为原型)\nmulti-headattention：patch-embedding的输出作为q查询，prototypes为k，v，全连接层让输出改为合适的形状\npap（prompt-as-prefix）：提示词放到数据的前边\n\nYOLO算法：\n指标分析：\nmap：\n\nIOU：交并比\n专业技能：\nword2vec：\n\nCBOW连续词袋模型（用于预测词语n）：\n\nn+2，n+1，n-1，n-2的独热码与embedding层相乘，得到形状为4128的向量为特征向量取平均得到1128的n的特征向量，通过线性层计算的到输出向量，再经过softmax函数得到目标词（n）的独热码，作为预测结果\n以下为实现代码（这里没有设置softmax）：\n\n跳字模型（skip-gram）：\nskip模型在迭代的过程中，目标词的词向量会向上下文的词向量靠近\n衡量两个词向量是否相似：使用词向量的点积：\n$$\n\\mathbf{a}\\cdot\\mathbf{b}=\\sum_{i=1}^{n}a_{i}b_{i}\n$$\n理解：$\\mathbf{a}\\cdot\\mathbf{b}=|A|\\cdot|B|\\cdot\\cos\\alpha$，$\\cos\\alpha$最大，就最相近，为1时就是$A\\cdotB$\n\nGloVe：\n思想：\n一个是基于奇异值分解SVD的lSA算法。\n', '_{i=1}^{n}a_{i}b_{i}\n$$\n理解：$\\mathbf{a}\\cdot\\mathbf{b}=|A|\\cdot|B|\\cdot\\cos\\alpha$，$\\cos\\alpha$最大，就最相近，为1时就是$A\\cdotB$\n\nGloVe：\n思想：\n一个是基于奇异值分解SVD的lSA算法。\n另

In [25]:
print(document_chunks[0])
print(document_chunks[1])

金融序列预测

挑战与任务：时间序列与自然语言对齐
该方法与传统训练方法比较：

模型介绍：

instanceNorm
patching：获取每一小部分的时间序列窗口，移动窗口，聚合局部特征
patchembedder：批处理向量化
textprototypes：先将预训练好的embedding与linear连接在一起，得到形状更小的embedding(prototypes翻译为原型)
multi-headattention：patch-embedding的输出作为q查询，prototypes为k，v，全连接层让输出改为合适的形状
pap（prompt-as-prefix）：提示词放到数据的前边

YOLO算法：
指标分析：
map：

IOU：交并比
专业技能：
word2vec：

CBOW连续词袋模型（用于预测词语n）：

n+2，n+1，n-1，n-2的独热码与embedding层相乘，得到形状为4128的向量为特征向量取平均得到1128的n的特征向量，通过线性层计算的到输出向量，再经过softmax函数得到目标词（n）的独热码，作为预测结果
以下为实现代码（这里没有设置softmax）：

跳字模型（skip-gram）：
skip模型在迭代的过程中，目标词的词向量会向上下文的词向量靠近
衡量两个词向量是否相似：使用词向量的点积：
$$
\mathbf{a}\cdot\mathbf{b}=\sum_{i=1}^{n}a_{i}b_{i}
$$
理解：$\mathbf{a}\cdot\mathbf{b}=|A|\cdot|B|\cdot\cos\alpha$，$\cos\alpha$最大，就最相近，为1时就是$A\cdotB$

GloVe：
思想：
一个是基于奇异值分解SVD的lSA算法。

_{i=1}^{n}a_{i}b_{i}
$$
理解：$\mathbf{a}\cdot\mathbf{b}=|A|\cdot|B|\cdot\cos\alpha$，$\cos\alpha$最大，就最相近，为1时就是$A\cdotB$

GloVe：
思想：
一个是基于奇异值分解SVD的lSA算法。
另一个方法是word2vec算法，该算法可以分为skip-gram和continuousbag-of-words（CBOW）两类。一个是利用了全局特征的矩阵分解方法，一个

# 4. 词向量数据库与向量检索模块

为了构建向量数据库，需要一下关键功能：

1. **持久化存储（persist）：** 将数据库存储到本地，便于此次加载使用
2. **加载数据库（load_vector）：** 从本地文件加载已经存储的向量和文档。
3. **获取向量表示（get_vector）：** 将文本转化为向量表示并存储
4. **检索（query）:** 根据用户的Query，检索数据库中的想啊滚文档片段

In [39]:
class VectorStore:
    def __init__(self, document: List[str] = None) -> None:
        """
        初始化向量存储类，存储文档和对应的向量表示
        param document：文档列表，默认为空
        """
        if document is None:
            documnt = []
        self.document = document # 存储文档内容
        self.vector = [] # 存储文档的向量表示

    def get_vector(self, EmbeddingModel:BaseEmbeddings) -> List[List[float]]:
        """
        使用传入的 Embedding 模型当文档向量化
        param EmbeddingMode： 传入的用于生成向量的模型（需继承 BaseEmbedings 类）
        return 返回文档对应的向量列表 
        """
        # 遍历所有文档，获取每个文档的向量表示
        self.vectors = [EmbeddingModel.get_embedding(doc) for doc in self.document]
        return self.vectors
    
    def persist(self, path:str='storage'):
        """
        将文本和对应的向量表示持久化本地目录中，以便后续加载使用
        param path：存储路径，默认为：storage
        """
        if not os.path.exists(path):
            os.makedirs(path) # 如果路径不存在，创建路径
        # 保存向量为 numpy 文件
        np.save(os.path.join(path, 'vectors.npy'), self.vectors)
        # 将文档内容存储到文本文件中
        with open(os.path.join(path, 'documents.txt'), 'w') as f:
            for doc in self.document:
                f.write(doc + '\n')

    def load_vector(self, path:str='storage'):
        # 加载保存的向量数据
        self.vectors = np.load(os.path.join(path, 'vectors.npy')).tolist()
        # 加载文档内容
        with open(os.path.join(path, 'documents.txt'), 'r') as f:
            self.document = [line.strip() for line in f.readlines()]

    def get_similarity(self, vector1:List[float], vector2:List[float]) -> float:
        """
        计算两个向量的预选相似度
        param vector1: 第一个向量
        param vector2: 第二个向量
        return: 返回两个向量的余弦相似度 -1到1
        """
        dot_product = np.dot(vector1, vector2)
        magnitude = np.linalg.norm(vector1) * np.linalg.norm(vector2)
        return dot_product / magnitude

    def query(self, query:str, EmbeddingModel:BaseEmbeddings, k:int = 1)->List[str]:
        """
        根据用户的查询文本，检索最相关的文档片段
        param query: 用户查询的文本
        param EmbeddingModel: 用于生成查询文本向量的模型
        param k: 返回最相关的 k 个文档片段，默认为1
        return: 返回最相关的 k 个文档片段
        """
        # 将查询文本向量化
        query_vector = EmbeddingModel.get_embedding(query)
        # 计算查询向量与每个文档向量的相似度
        similarities = [self.get_similarity(query_vector, vec) for vec in self.vectors]
        # 获取最相关的 k 个文档片段
        top_k_indices = np.argsort(similarities)[-k:][::-1]
        # 返回对应的文档内容
        return [self.document[idx] for idx in top_k_indices]

In [29]:
# 测试
documents = [
    "机器学习是人工智能的一个分支。",
    "深度学习是一种特殊的机器学习方法",
    "监督学习是一种训练模型的方法",
    "强化学习是通过奖励和惩罚进行学习。",
    "无监督学习不依赖标签数据"
]

In [40]:
# 创建向量数据库
vector_store = VectorStore(documents)

# 使用 OpenAI Embedding 模型对文档进行向量化
embedding_model = OpenAIEmbedding()

# 获取文档向量并存储
vector_store.get_vector(embedding_model)

# 持久化存储到本地
vector_store.persist('storage')

In [41]:
# 模拟用户查询
query = "什么是深度学习？"
result = vector_store.query(query, embedding_model)

print('检索结果：', result)

检索结果： ['深度学习是一种特殊的机器学习方法']


In [42]:
loaded_array = np.load('storage/vectors.npy')
print('加载的向量：', loaded_array)

加载的向量： [[-0.02901346 -0.03527604 -0.03944485 ...  0.00833763  0.02512506
   0.02052628]
 [-0.05525073 -0.05735225 -0.03717771 ...  0.00734097  0.02640267
   0.00717381]
 [-0.06147213 -0.03349644 -0.04561467 ... -0.00824196  0.0192443
   0.03054029]
 [-0.07408334 -0.01950175 -0.01650073 ... -0.01323749  0.00274608
   0.01729712]
 [-0.09506936 -0.01732405 -0.04136577 ...  0.00466926 -0.02933072
   0.03368302]]


# 5. 大模型问答模块

先创建基类`BaseModel`，然后再以 GPT4o 模型为例，展示如何使用大语言模型里完成这个任务

### 5.1 大模型模块的基类
 - chat：负责处理用户的输入并生成回答
 - load_model：如果是使用本地模型，这个方法负责加载模型。如果使用API模型，可以不用实现这个方法。 

In [43]:
class BaseModel:
    """
    基础模型类，作为所有模型的基类
    包含一些通用的接口，如加载模型、生成回答等
    """
    def __init__(self, path:str='') -> None:
        self.path = path

    def chat(self, prompt:str, history:List[dict], content:str) -> str:
        """
        处理用户的输入并生成回答
        param query: 用户输入的查询
        return: 模型生成的回答
        """
        raise NotImplementedError

    def load_model(self) -> None:
        """
        加载模型
        """
        raise NotImplementedError

### 5.2 借助GPT4o模型进行对话

In [62]:
class GPT4oChat(BaseModel):
    """
    GPT4o模型对话类
    """
    def __init__(self, api_key:str=api_key
                 , base_url:str=base_url) -> None:
        super().__init__()
        self.client = OpenAI(api_key=api_key, base_url=base_url)

    def chat(self, prompt:str, history:List=[], content:str='') -> str:
        """
        处理用户的输入并生成回答
        param prompt：用户的提问
        param query: 用户输入的查询（可选）
        param content：可参考的上下文信息（可选）
        return：生成的回答
        """
        # 构建包含问题和上下文的完整提示
        full_prompt = PROMPT_TEMPLATE['GPT4o_PROMPT_TEMPLATE'].format(question=prompt, context=content)

        # 调用 GPT-4o 模型进行推理
        response = self.client.chat.completions.create(
            model='deepseek-r1',
            messages=[
                {"role": "user", "content": full_prompt}
            ]
        )

        # 返回模型生成的第一个答案
        return response.choices[0].message.content



### 5.3 提示模版
为了方便维护和复用提示词，可以使用一个字典来保存不同模型的提示模版

In [63]:
PROMPT_TEMPLATE = dict(
    GPT4o_PROMPT_TEMPLATE = """
下面有一个或许与这个问题相关的参考段落，若你觉得参考段落能和问题相关，这现总结参考段落的内容。
若你觉得参考段落和问题无关，这使用你自己的原始知识来回答用户的问题，并且送死使用中文来回答
问题：{question}
可参考的上下文
···
{context}
···
有用的回答：
"""
)

# 6. 完整流程演示

In [64]:
# 加载并切分文档
docs = ReadFiles('./data').get_content(max_token_len=600, cover_content=150)
vector = VectorStore(docs)

In [65]:
# 使用OpenAI Embedding 模型惊进行向量化
embedding = OpenAIEmbedding()
vector.get_vector(EmbeddingModel=embedding)

[[-0.07588373124599457,
  -0.00533860269933939,
  -0.053754206746816635,
  -0.014068332500755787,
  -0.05096379667520523,
  -0.0003348737955093384,
  0.025210607796907425,
  0.05224273353815079,
  0.025481898337602615,
  0.03131463751196861,
  -0.05697093531489372,
  0.004127486143261194,
  -0.006041050422936678,
  0.03422131761908531,
  0.021664457395672798,
  0.01308975089341402,
  -0.011878633871674538,
  -0.0004578021471388638,
  -0.005760071333497763,
  -0.09433146566152573,
  0.01986231654882431,
  0.006675675977021456,
  0.018312085419893265,
  0.02189699187874794,
  0.012808771803975105,
  0.07382968068122864,
  -0.09301377087831497,
  -0.05011117085814476,
  0.037922490388154984,
  0.014194289222359657,
  0.01729474775493145,
  0.05642835423350334,
  0.026625191792845726,
  -0.04848342761397362,
  -0.02941560558974743,
  0.021548191085457802,
  0.007407190278172493,
  0.015715451911091805,
  -0.0777827650308609,
  0.021141255274415016,
  0.01990107074379921,
  0.01179143320769

In [66]:
# 将向量和文本保存到本地
vector.persist(path = 'storage')
# 用户提出问题
question = 'RAG技术的基本原理是什么？'
# 在数据库中检索最相关的文档片段
content = vector.query(question, EmbeddingModel=embedding, k=1)[0]

In [67]:
content

'调和平均值\n\nRAG技术基本原理\n前期问题：大模型无法处理长文本问题\n更新大模型知识库设计：\n\n微调模型：问题在于效果不稳定，少量文本容易被遗忘；成本高昂\n增强模型上下文长度：GLM4-long\n知识检索：提取文本一部分的内容，结合提示词，把知识库的相关文档片段“检索出来”\n\n检索技术发展迭代：\n\n长文本切分成小文本，将用户问题短文档进行匹配，找到最相关的topk文档，发送给大模型引导用户回答相关问题微软的RAGgraph\n\nRAG技术核心步骤与通用优化方法：\n\n技术难点：每个步骤的技术优化\nRAG实践落地项目：\n\n手动搭建\n使用langchain、Llama-index等开源项目快速搭建\n使用GLM、openAIAssistantAPI等快速实现\n'

In [68]:
# 使用 GPT40Chat 模型生成答案
chat = GPT4oChat()
print(chat.chat(question, [], content)) 

根据提供的参考段落，RAG（Retrieval-Augmented Generation）技术的基本原理可总结如下：

**RAG的核心原理**  
通过结合检索（Retrieval）与生成（Generation）技术，解决大模型处理长文本时存在的知识更新困难和上下文长度限制问题。其核心步骤为：  
1. **知识检索**：将长文本切分为小片段，根据用户问题检索出最相关的Top-K文档片段；  
2. **增强生成**：将检索到的内容结合提示词输入大模型，引导模型生成更准确的回答。

**关键技术设计**  
- **知识更新优化**：避免直接微调模型（成本高、效果不稳定），转而通过外部知识库动态检索关联内容。  
- **检索技术迭代**：如微软的RAG Graph通过文档间关联关系优化检索路径。  
- **工程实践**：可手动搭建或利用LangChain、Llama Index等开源框架快速实现，或调用GLM/OpenAI等API集成。

**优势与价值**  
RAG在成本、知识更新灵活性和长文本处理能力（如GLM4-long扩展上下文）上优于传统微调方法，成为当前大模型落地的关键技术之一。


In [69]:
def run_mini_rag(question:str, knowledge_base_path:str, k:int=1) -> str:
    """
    运行一个简化版的RAG项目

    param question: 用户提出的问题
    param knowledge_base_path：知识库的路径，包含文档的文件夹路径
    param api_key：OpenAI API密钥，用于调用GPT-4o模型
    param k：返回与文艺最想概念的k个文档片段，默认为1
    return：返回GPT-4o模型生成的回答
    """
    api_key = os.getenv('OPENAI_API_KEY')
    # 1. 加载并切分文档
    docs = ReadFiles(knowledge_base_path).get_content(max_token_len=600, cover_content=150)
    vector = VectorStore(docs)
    # 2. 使用OpenAI Embedding 模型惊进行向量化
    embedding = OpenAIEmbedding()
    vector.get_vector(EmbeddingModel=embedding)
    # 3. 将向量和文本保存到本地（可选）
    vector.persist(path = 'storage')
    # 4. 在数据库中检索最相关的文档片段
    content = vector.query(question, EmbeddingModel=embedding, k=k)[0]
    # 5. 使用GPT-4o模型生成答案
    chat = GPT4oChat(api_key=api_key)
    answer = chat.chat(question, [], content)

    return answer



In [70]:
question = 'RAG技术的基本原理是什么？'
knowledge_base_path = './data'
answer = run_mini_rag(question, knowledge_base_path)

In [71]:
display(Markdown(answer))

RAG（Retrieval-Augmented Generation，检索增强生成）技术是一种通过结合检索与生成能力来增强大语言模型（如GPT、GLM等）效果的方法，其基本原理可概括为以下核心步骤和设计逻辑：

---

### **1. 核心目标**
解决大模型在处理长文本时的局限性：
- **长文本处理困难**：传统模型对超长上下文理解能力有限（如早期模型的输入长度限制）。
- **知识更新成本高**：直接微调模型以适应新知识可能导致效果不稳定且成本高昂。

---

### **2. 基本原理**
RAG通过动态检索外部知识库来补充模型的输入，分三步实现：
1. **文本切分与索引**  
   将长文本切分为小片段（如段落或句子），构建向量化索引库，便于快速检索。

2. **相关性检索**  
   根据用户问题，从知识库中检索最相关的文档片段（如通过语义相似度匹配，返回Top-K结果）。例如，使用微软的RAG Graph等技术优化检索路径和效率。

3. **生成增强**  
   将检索到的相关片段与用户问题结合，通过提示词（Prompt）引导模型生成更准确的回答，而非仅依赖模型内部记忆。

---

### **3. 技术优势**
- **动态知识扩展**：无需频繁微调模型，即可实时更新知识库。
- **成本效益**：相比模型微调，检索外部知识的资源消耗更低。
- **效果稳定性**：通过聚焦相关上下文，减少模型生成无关或错误内容的概率。

---

### **4. 关键优化方向**
- **检索效率**：优化文本分块策略、索引算法（如使用稠密向量检索）。
- **上下文整合**：设计更有效的提示模板，协调检索内容与生成逻辑。
- **工具支持**：借助LangChain、LlamaIndex等框架快速搭建，或通过GLM、OpenAI Assistant API等集成实现。

---

### **总结**
RAG通过“检索+生成”的协同机制，将外部知识库的动态检索与大模型的生成能力结合，既弥补了模型处理长文本和实时更新的不足，又降低了依赖微调的成本，成为当前增强大模型实用性的主流技术之一。